In [3]:
import pandas as pd
import numpy as np
from itertools import product
import plotly.graph_objs as go

In [4]:
class MeanReversionBacktester():

  def __init__(self, symbol, SMA, dev, start, end, tc):

    self.symbol = symbol
    self.SMA = SMA
    self.dev = dev
    self.start = start
    self.end = end
    self.tc = tc
    self.results = None
    self.get_data()
    self.prepare_data()

  def __repr__(self):

    return f'MeanReversionBacktester(symbol = {self.symbol}, SMA = {self.SMA}, dev = {self.dev}, start = {self.start}, end = {self.end}, tc={self.tc})'

  def get_data(self):

    if self.symbol == 'EUR/USD':

      raw = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')
      raw = raw.Close.to_frame().dropna()
      raw = raw[self.start:self.end]
      raw["returns"] = np.log(raw / raw.shift(1))
      self.data = raw

    # Maybe define an else case to get data from yahoo finance

  def prepare_data(self):

    data = self.data.copy()
    data['SMA'] = data['Close'].rolling(self.SMA).mean()
    data['Lower'] = data['SMA'] - data['Close'].rolling(self.SMA).std() * self.dev
    data['Upper'] = data['SMA'] + data['Close'].rolling(self.SMA).std() * self.dev
    data.dropna(inplace=True)
    self.data = data

  def set_parameters(self, SMA = None, dev = None):
        
    if SMA is not None:
        self.SMA = SMA
        self.data["SMA"] = self.data["Close"].rolling(self.SMA).mean()
        self.data["Lower"] = self.data["SMA"] - self.data["Close"].rolling(self.SMA).std() * self.dev
        self.data["Upper"] = self.data["SMA"] + self.data["Close"].rolling(self.SMA).std() * self.dev
        
    if dev is not None:
        self.dev = dev
        self.data["Lower"] = self.data["SMA"] - self.data["Close"].rolling(self.SMA).std() * self.dev
        self.data["Upper"] = self.data["SMA"] + self.data["Close"].rolling(self.SMA).std() * self.dev

  def test_strategy(self):

    data = self.data.copy().dropna()
    data["distance"] = data.Close - data.SMA
    data["position"] = np.where(data.Close < data.Lower, 1, np.nan)
    data["position"] = np.where(data.Close > data.Upper, -1, data["position"])
    data["position"] = np.where(data.distance * data.distance.shift(1) < 0, 0, data["position"])
    data["position"] = data.position.ffill().fillna(0)
    data["strategy"] = data.position.shift(1) * data["returns"]
    data.dropna(inplace=True)
    
    # determine the number of trades in each bar
    data["trades"] = data.position.diff().fillna(0).abs()
    
    # subtract transaction/trading costs from pre-cost return
    data['strategy_net'] = data.strategy - data.trades * self.tc
    
    data["creturns"] = data["returns"].cumsum().apply(np.exp)
    data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
    data["cstrategy_net"] = data["strategy_net"].cumsum().apply(np.exp)
    self.results = data
    
    perf = data["cstrategy"].iloc[-1] # absolute performance of the strategy
    outperf = perf - data["creturns"].iloc[-1] # out-/underperformance of strategy
    
    return round(perf, 6), round(outperf, 6)

  def plot_results(self):

    if self.results is None:
      print("Run test_strategy() first")

    else:
      fig = go.Figure()

      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.creturns, name='Returns (Base)'))
      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.cstrategy, name='Returns (Strategy)'))
      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.cstrategy_net, name='Returns (Strategy + cost)'))

      title = f"{self.symbol} | SMA = {self.SMA} | dev = {self.dev} | TC = {self.tc}"
      fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price')

      fig.show()

  def optimize_parameters(self, SMA_range, dev_range):

    combinations = list(product(range(*SMA_range), range(*dev_range)))
    
    # test all combinations
    results = []
    for comb in combinations:
        self.set_parameters(comb[0], comb[1])
        results.append(self.test_strategy()[0])
    
    best_perf = np.max(results) # best performance
    opt = combinations[np.argmax(results)] # optimal parameters
    
    # run/set the optimal strategy
    self.set_parameters(opt[0], opt[1])
    self.test_strategy()
                
    # create a df with many results
    many_results =  pd.DataFrame(data = combinations, columns = ["SMA", "dev"])
    many_results["performance"] = results
    self.results_overview = many_results
                        
    return opt, best_perf

In [5]:
obj = MeanReversionBacktester(symbol="EUR/USD", SMA=30, dev=2, start='2010', end='2020', tc=0.00007)

In [6]:
obj

MeanReversionBacktester(symbol = EUR/USD, SMA = 30, dev = 2, start = 2010, end = 2020, tc=7e-05)

In [7]:
obj.data

,Close,returns,SMA,Lower,Upper
time,,,,,
2010-02-11,1.368176,-0.005213,1.413171,1.355739,1.470602
2010-02-12,1.363494,-0.003428,1.410654,1.351320,1.469989
2010-02-15,1.360304,-0.002342,1.407918,1.347088,1.468748
2010-02-16,1.376709,0.011988,1.405922,1.345055,1.466788
2010-02-17,1.360692,-0.011702,1.403264,1.341672,1.464857
...,...,...,...,...,...
2020-12-25,1.218472,-0.000549,1.204852,1.176300,1.233405
2020-12-28,1.220510,0.001671,1.206052,1.178014,1.234089
2020-12-29,1.222345,0.001502,1.207265,1.179681,1.234849


In [8]:
obj.test_strategy()

(0.941526, 0.042527)

In [9]:
obj.results

,Close,returns,SMA,Lower,Upper,distance,position,strategy,trades,strategy_net,creturns,cstrategy,cstrategy_net
time,,,,,,,,,,,,,
2010-02-12,1.363494,-0.003428,1.410654,1.351320,1.469989,-0.047160,0.0,-0.000000,0.0,-0.000000,0.996578,1.000000,1.000000
2010-02-15,1.360304,-0.002342,1.407918,1.347088,1.468748,-0.047614,0.0,-0.000000,0.0,-0.000000,0.994246,1.000000,1.000000
2010-02-16,1.376709,0.011988,1.405922,1.345055,1.466788,-0.029213,0.0,0.000000,0.0,0.000000,1.006237,1.000000,1.000000
2010-02-17,1.360692,-0.011702,1.403264,1.341672,1.464857,-0.042572,0.0,-0.000000,0.0,-0.000000,0.994530,1.000000,1.000000
2010-02-18,1.346149,-0.010746,1.400409,1.336397,1.464421,-0.054261,0.0,-0.000000,0.0,-0.000000,0.983900,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,1.218472,-0.000549,1.204852,1.176300,1.233405,0.013620,-1.0,0.000549,0.0,0.000549,0.890581,0.950426,0.940696
2020-12-28,1.220510,0.001671,1.206052,1.178014,1.234089,0.014458,-1.0,-0.001671,0.0,-0.001671,0.892070,0.948839,0.939126
2020-12-29,1.222345,0.001502,1.207265,1.179681,1.234849,0.015080,-1.0,-0.001502,0.0,-0.001502,0.893412,0.947415,0.937716


In [10]:
obj.results.trades.value_counts()

0.0    2688
1.0     147
Name: trades, dtype: int64

In [11]:
obj.plot_results()

In [12]:
SMA_range = (10, 150)
dev_range = (1, 3)

In [13]:
obj.optimize_parameters(SMA_range=SMA_range, dev_range=dev_range)

((10, 1), 1.4544)

In [14]:
obj_optimized = MeanReversionBacktester(symbol="EUR/USD", SMA=58, dev=1, start='2018', end='2020', tc=0.00007)

In [15]:
obj_optimized.plot_results()

Run test_strategy() first


In [16]:
obj.test_strategy()

(1.4544, 0.546201)

In [17]:
obj.plot_results()